## Introduction

(Sorry for probably existing mistakes, my native language is Russian and this course is some kind of challenge for me)

I’m not sure if it is exactly a business problem, but I want to compare my hometown – Voronezh with my new city – Moscow. Voronezh is a quite developed city with more than one million population. Obviously, life there is less expensive than in capital city. But this project gave me interesting idea – I can compare these two places only from venue point of view. Maybe in this particular case there is no such terrific difference. Or maybe all neighborhoods of Voronezh closer to only suburban areas of Moscow, who knows? 

Let’s check!

And from a business perspective it’s useful to gain such a comparison in case of choosing more or less similar starting point  for entrepreneur from Voronezh to expand his business and conquer the capital city!


## Data
Let’s talk about data.

Obviously, I’ll use foursquare API for gaining venue data. 
For neighborhoods I’ll use so called ‘administrative districts’.
There are ten of them in Moscow and likely I’ll have to geocode them. 

That’s it, that’s all I need to start work.


In [1]:
import pandas as pd
from geopandas.tools import geocode
import geopy
import numpy as np
from sklearn.cluster import KMeans
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests
import json
from pandas.io.json import json_normalize

In [2]:

geopy.geocoders.options.default_user_agent = "mb.mozikov@gmail.com"

In [3]:
CLIENT_ID = 'E0VRZ503WXG1AXTVACAFDH4WTRBN15FJ1H4AWE4JUQZJ4YJD' # your Foursquare ID
CLIENT_SECRET = 'HQJVJGAWXMUZKTEUIE4B3MK1GCBQDDI0HKDWPYHLSFZPYGG0' # your Foursquare Secret
VERSION = '20201231' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

foursquare credentails:
CLIENT_ID: E0VRZ503WXG1AXTVACAFDH4WTRBN15FJ1H4AWE4JUQZJ4YJD
CLIENT_SECRET:HQJVJGAWXMUZKTEUIE4B3MK1GCBQDDI0HKDWPYHLSFZPYGG0


#### We begin with Voronezh

In [4]:
latitude, longitude = 51.67204, 39.1843 #VRH coords
map_vrh = folium.Map(location=[latitude, longitude], zoom_start=10)

In [5]:
LIMIT = 120
radius = 4000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=E0VRZ503WXG1AXTVACAFDH4WTRBN15FJ1H4AWE4JUQZJ4YJD&client_secret=HQJVJGAWXMUZKTEUIE4B3MK1GCBQDDI0HKDWPYHLSFZPYGG0&v=20201231&ll=51.67204,39.1843&radius=4000&limit=120'

In [6]:
vrh_districts = ['Железнодорожный район', 'Коминтерновский район', 'Левобережный район',
                 'Ленинский район', 'Советский район', 'Центральный район']
vrh_districts_df = pd.DataFrame(columns=['geometry', 'address'])
for dist in vrh_districts:
    result = geocode('Россия, Воронеж, {}'.format(dist), provider="nominatim")
    vrh_districts_df = pd.concat([result, vrh_districts_df])
vrh_districts_df.reset_index(inplace=True)
vrh_districts_df.drop('index', axis=1, inplace=True)

In [7]:
points = vrh_districts_df.iloc[:,0]
longs = []
lats = []
for dot in points:
    longs.append(dot.x)
    lats.append(dot.y)
    
vrh_districts_df['Latitude'] = lats
vrh_districts_df['Longitude'] = longs
vrh_districts_df.drop('geometry', axis=1, inplace=True)
vrh_districts_df

,address,Latitude,Longitude
0,"Центральный район, городской округ Воронеж, Во...",51.733033,39.212859
1,"Советский район, городской округ Воронеж, Воро...",51.627819,39.114053
2,"Ленинский район, городской округ Воронеж, Воро...",51.642141,39.184415
3,"Левобережный район, городской округ Воронеж, В...",51.580273,39.265007
4,"Коминтерновский район, городской округ Воронеж...",51.718921,39.146199
5,"Железнодорожный район, городской округ Воронеж...",51.766379,39.348958


In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=2000):  # 500 meters not quite a radius for a neighborhood 
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
lists = vrh_districts_df['address'].str.split(',')
for i in range(len(lists)):
    vrh_districts_df.iloc[i,0] = str(lists[i][0])

In [10]:
venues_to_explore_vrh = getNearbyVenues(vrh_districts_df['address'],
                                        vrh_districts_df['Latitude'],
                                        vrh_districts_df['Longitude'])

Центральный район
Советский район
Ленинский район
Левобережный район
Коминтерновский район
Железнодорожный район


In [11]:
venues_to_explore_vrh

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Центральный район,51.733033,39.212859,Дендрарий,51.717482,39.220091,Garden
1,Центральный район,51.733033,39.212859,The Voda,51.725491,39.235660,Recreation Center
2,Центральный район,51.733033,39.212859,Добрый,51.727622,39.201500,Grocery Store
3,Центральный район,51.733033,39.212859,Санаторий им. Горького,51.729223,39.227036,Resort
4,Центральный район,51.733033,39.212859,Пляж санатория М. Горького,51.729843,39.227610,Beach
...,...,...,...,...,...,...,...
127,Железнодорожный район,51.766379,39.348958,пляж Сосновый бор (платный),51.776305,39.345334,Beach
128,Железнодорожный район,51.766379,39.348958,Пляж на Усманке,51.779800,39.356205,Beach
129,Железнодорожный район,51.766379,39.348958,Болото Гадючье,51.757014,39.368260,Lake
130,Железнодорожный район,51.766379,39.348958,"ДОЛ ""Зеленый Огонек""",51.755956,39.366913,Campground


In [12]:
len(venues_to_explore_vrh['Venue Category'].unique())

70

#### Now we have some a bit raw data about Voronezh
Let's proceed to Moscow

Moscow is about 15 times bigger than Voronezh and above 'neighborhoods' or districts has Administrative districts. There are 10 of them. First of all, I explore them.

In [13]:
moscow_adm_dist = ['Центральный', 'Северный', 'Северо-Восточный', 'Восточный',
                   'Юго-Восточный', 'Южный', 'Юго-Западный', 'Западный',
                   'Северо-Западный', 'Зеленоградский']
msk_adm_districts_df = pd.DataFrame(columns=['geometry', 'address'])
for dist in moscow_adm_dist:
    result = geocode('Россия, Москва, {} Административный Округ'.format(dist), provider="nominatim")
    msk_adm_districts_df = pd.concat([result, msk_adm_districts_df])
msk_adm_districts_df.reset_index(inplace=True)
msk_adm_districts_df.drop('index', axis=1, inplace=True)


In [14]:
points = msk_adm_districts_df.iloc[:,0]
longs = []
lats = []
for dot in points:
    longs.append(dot.x)
    lats.append(dot.y)
    
msk_adm_districts_df['Latitude'] = lats
msk_adm_districts_df['Longitude'] = longs
msk_adm_districts_df.drop('geometry', axis=1, inplace=True)
msk_adm_districts_df

,address,Latitude,Longitude
0,"Зеленоградский административный округ, Москва,...",55.984968,37.205612
1,"Северо-Западный административный округ, Москва...",55.826742,37.435495
2,"Западный административный округ, Москва, Центр...",55.700349,37.467540
3,"Юго-Западный административный округ, Москва, Ц...",55.602850,37.540927
4,"Южный административный округ, Москва, Ленински...",55.647965,37.644083
5,"Юго-Восточный административный округ, Москва, ...",55.699335,37.765248
6,"Восточный административный округ, Балашиха, го...",55.786904,37.752727
7,"Северо-Восточный административный округ, Москв...",55.870168,37.643606
8,"Северный административный округ, Москва, Центр...",55.833657,37.525809
9,"Центральный административный округ, Москва, Це...",55.753848,37.595464


In [15]:
#There is a mistake in work of geocoder, I'll correct it manually 
msk_adm_districts_df.iloc[6,1], msk_adm_districts_df.iloc[6,2] = 55.787710, 37.775631

In [16]:
msk_adm_districts_df

,address,Latitude,Longitude
0,"Зеленоградский административный округ, Москва,...",55.984968,37.205612
1,"Северо-Западный административный округ, Москва...",55.826742,37.435495
2,"Западный административный округ, Москва, Центр...",55.700349,37.467540
3,"Юго-Западный административный округ, Москва, Ц...",55.602850,37.540927
4,"Южный административный округ, Москва, Ленински...",55.647965,37.644083
5,"Юго-Восточный административный округ, Москва, ...",55.699335,37.765248
6,"Восточный административный округ, Балашиха, го...",55.787710,37.775631
7,"Северо-Восточный административный округ, Москв...",55.870168,37.643606
8,"Северный административный округ, Москва, Центр...",55.833657,37.525809
9,"Центральный административный округ, Москва, Це...",55.753848,37.595464


In [17]:
lists = msk_adm_districts_df['address'].str.split(',')
for i in range(len(lists)):
    msk_adm_districts_df.iloc[i,0] = str(lists[i][0])

In [18]:
msk_adm_districts_df

,address,Latitude,Longitude
0,Зеленоградский административный округ,55.984968,37.205612
1,Северо-Западный административный округ,55.826742,37.435495
2,Западный административный округ,55.700349,37.467540
3,Юго-Западный административный округ,55.602850,37.540927
4,Южный административный округ,55.647965,37.644083
5,Юго-Восточный административный округ,55.699335,37.765248
6,Восточный административный округ,55.787710,37.775631
7,Северо-Восточный административный округ,55.870168,37.643606
8,Северный административный округ,55.833657,37.525809
9,Центральный административный округ,55.753848,37.595464


In [19]:
venues_to_explore_msk = getNearbyVenues(msk_adm_districts_df['address'],
                                        msk_adm_districts_df['Latitude'],
                                        msk_adm_districts_df['Longitude'], radius=4000)
#bigger radius for bigges adm. elements

Зеленоградский административный округ
Северо-Западный административный округ
Западный административный округ
Юго-Западный административный округ
Южный административный округ
Юго-Восточный административный округ
Восточный административный округ
Северо-Восточный административный округ
Северный административный округ
Центральный административный округ


In [20]:
venues_to_explore_msk['Venue Category'].value_counts()

Park                    85
Gym / Fitness Center    59
Coffee Shop             44
Health Food Store       34
Cosmetics Shop          27
                        ..
Car Wash                 1
Dairy Store              1
Resort                   1
Scenic Lookout           1
Lounge                   1
Name: Venue Category, Length: 195, dtype: int64

There are way too much unique entries. First, we cut off less frequent

In [21]:
less_freq_venues = venues_to_explore_msk['Venue Category'].value_counts()[venues_to_explore_msk['Venue Category'].value_counts() < 2].index

In [22]:
venues_to_explore_msk.index = venues_to_explore_msk['Venue Category']
venues_to_explore_msk_cut = venues_to_explore_msk.drop(less_freq_venues).drop('Venue Category', axis=1).reset_index()

In [23]:
venues_to_explore_msk_cut['Venue Category'].unique()

array(['Athletics & Sports', 'Trail', 'Park', 'Pizza Place',
       'Food & Drink Shop', 'Bar', 'Dessert Shop', 'Gym / Fitness Center',
       'Beer Bar', 'Beach', 'Pool', 'Forest', 'American Restaurant',
       'Concert Hall', 'Donut Shop', 'Stables', 'Theater', 'Soccer Field',
       'Bakery', 'Brewery', 'Flower Shop', 'Climbing Gym', 'Pharmacy',
       'Gym', 'Plaza', 'Hookah Bar', 'Gourmet Shop', 'Movie Theater',
       'Cosmetics Shop', 'Racetrack', 'Sports Club', 'Asian Restaurant',
       'History Museum', 'Café', 'Japanese Restaurant', 'Coffee Shop',
       'Cupcake Shop', 'Go Kart Track', 'Health & Beauty Service',
       'Fast Food Restaurant', 'Pool Hall', 'Clothing Store',
       'Sporting Goods Shop', 'Fish Market', 'Supermarket', 'Restaurant',
       'Shopping Mall', 'Pub', 'Mobile Phone Shop', 'Burger Joint',
       'Arcade', 'Auto Workshop', 'Tennis Court', 'Hockey Arena',
       'Farmers Market', 'Gastropub', 'Soccer Stadium', 'Surf Spot',
       'Furniture / Home Stor

In [24]:
from functools import partial
def generalize(el, word='Restaurant'):
    temp = str(el).split(' ')
    if word in temp:
        return word
    else:
        return el

In [25]:
reduce_lst = ['Restaurant', 'Bar', 'Pizza', 'Gym', 'Sports', 'Stadium', 'Yoga', 'Dance', 'Museum']
for elt in reduce_lst:
    t_gen = partial(generalize, word=elt)
    venues_to_explore_msk_cut['Venue Category'] = venues_to_explore_msk_cut['Venue Category'].apply(t_gen)

In [26]:
venues_to_explore_msk_cut['Venue Category'].value_counts()

Park                  85
Gym                   78
Restaurant            71
Coffee Shop           44
Health Food Store     34
                      ..
Track                  2
Gun Range              2
Skate Park             2
Sauna / Steam Room     2
Buffet                 2
Name: Venue Category, Length: 112, dtype: int64

A little bit better

Now we'll do the same for Vrh

In [32]:
venues_to_explore_vrh

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Центральный район,51.733033,39.212859,Дендрарий,51.717482,39.220091,Garden
1,Центральный район,51.733033,39.212859,The Voda,51.725491,39.235660,Recreation Center
2,Центральный район,51.733033,39.212859,Добрый,51.727622,39.201500,Grocery Store
3,Центральный район,51.733033,39.212859,Санаторий им. Горького,51.729223,39.227036,Resort
4,Центральный район,51.733033,39.212859,Пляж санатория М. Горького,51.729843,39.227610,Beach
...,...,...,...,...,...,...,...
127,Железнодорожный район,51.766379,39.348958,пляж Сосновый бор (платный),51.776305,39.345334,Beach
128,Железнодорожный район,51.766379,39.348958,Пляж на Усманке,51.779800,39.356205,Beach
129,Железнодорожный район,51.766379,39.348958,Болото Гадючье,51.757014,39.368260,Lake
130,Железнодорожный район,51.766379,39.348958,"ДОЛ ""Зеленый Огонек""",51.755956,39.366913,Campground


Obviously, there is no such diversity as in Msk, but let's preprocess in the same way to work with same categories

In [30]:
reduce_lst = ['Restaurant', 'Bar', 'Pizza', 'Gym', 'Sports', 'Stadium', 'Yoga', 'Dance', 'Museum']
for elt in reduce_lst:
    t_gen = partial(generalize, word=elt)
    venues_to_explore_vrh['Venue Category'] = venues_to_explore_vrh['Venue Category'].apply(t_gen)

In [36]:
venues_oh_vrh = pd.get_dummies(venues_to_explore_vrh[['Venue Category']], prefix="", prefix_sep="")
venues_oh_vrh['Neighborhood'] = venues_to_explore_vrh['Neighborhood']
fixed_cols = [venues_oh_vrh.columns[-1]] + list(venues_oh_vrh.columns[:-1])
venues_oh_vrh = venues_oh_vrh[fixed_cols]
vrh_grouped = venues_oh_vrh.groupby('Neighborhood').mean().reset_index()


,Neighborhood,Supermarket,ATM,Airport,Art Gallery,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bath House,...,Restaurant,River,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Sporting Goods Shop,Sports,Stadium,Steakhouse
0,Железнодорожный район,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Коминтерновский район,0.047619,0.000000,0.000000,0.000000,0.000000,0.015873,0.047619,0.015873,0.000000,...,0.126984,0.000000,0.000000,0.031746,0.015873,0.000000,0.015873,0.015873,0.015873,0.000000
2,Ленинский район,0.022727,0.022727,0.000000,0.045455,0.045455,0.000000,0.045455,0.022727,0.022727,...,0.181818,0.000000,0.022727,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.022727
3,Советский район,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Центральный район,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [39]:
def return_most_common_venues(row, num_top_venues):
    #row_categories = row.iloc[1:]
    row_categories_sorted = row.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = vrh_grouped['Neighborhood']

for ind in np.arange(vrh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vrh_grouped.iloc[ind, 1:], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Железнодорожный район,Beach,Lake,Resort,Campground,Steakhouse,Bus Line,Diner,Dessert Shop,Deli / Bodega,Dance
1,Коминтерновский район,Restaurant,Pub,Park,Supermarket,Bakery,Clothing Store,Electronics Store,Grocery Store,Bookstore,Gym
2,Ленинский район,Restaurant,Bookstore,Grocery Store,Park,Food & Drink Shop,Gym,Bakery,Arts & Crafts Store,Art Gallery,Bar
3,Советский район,Airport,Restaurant,Bath House,Steakhouse,Bus Station,Electronics Store,Diner,Dessert Shop,Deli / Bodega,Dance
4,Центральный район,Bus Stop,Bus Line,Gym,Nature Preserve,Bistro,Beach,Recreation Center,Bus Station,Resort,River


In [43]:
venues_oh_msk = pd.get_dummies(venues_to_explore_msk_cut[['Venue Category']], prefix="", prefix_sep="")
venues_oh_msk['Neighborhood'] = venues_to_explore_msk_cut['Neighborhood']
fixed_cols = [venues_oh_msk.columns[-1]] + list(venues_oh_msk.columns[:-1])
venues_oh_msk = venues_oh_msk[fixed_cols]
msk_grouped = venues_oh_msk.groupby('Neighborhood').mean().reset_index()

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_msk = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_msk['Neighborhood'] = msk_grouped['Neighborhood']

for ind in np.arange(msk_grouped.shape[0]):
    neighborhoods_venues_sorted_msk.iloc[ind, 1:] = return_most_common_venues(msk_grouped.iloc[ind, 1:], num_top_venues)

neighborhoods_venues_sorted_msk.head(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Восточный административный округ,Health Food Store,Gym,Park,Restaurant,Coffee Shop,Candy Store,Supermarket,Dance,Photography Studio,Bar
1,Западный административный округ,Gym,Park,Restaurant,Coffee Shop,Auto Workshop,Gourmet Shop,Spa,Sports,Health Food Store,Yoga
2,Зеленоградский административный округ,Park,Gym,Restaurant,Bar,Coffee Shop,Café,Clothing Store,Brewery,Concert Hall,Pizza
3,Северный административный округ,Park,Gym,Coffee Shop,Restaurant,Cosmetics Shop,Health Food Store,Yoga,Pet Store,Clothing Store,Auto Workshop
4,Северо-Восточный административный округ,Park,Gym,Restaurant,Health Food Store,Grocery Store,Multiplex,Arts & Crafts Store,Cosmetics Shop,Bar,Supermarket


Let's concat Vrh and Msk df and do the clustering

In [50]:
concated = pd.concat([vrh_grouped, msk_grouped]).fillna(0)

In [52]:
num_clusters = 6
concated_clustering = concated.drop('Neighborhood', axis=1)

In [54]:
KM = KMeans(n_clusters=num_clusters, random_state=1)
KM.fit(concated_clustering)
KM.labels_[0:10]

array([3, 4, 4, 2, 0, 1, 5, 5, 5, 1])

In [70]:
len(KM.labels_)

15

In [67]:
data = pd.concat([vrh_districts_df, msk_adm_districts_df]).reset_index().drop('index', axis=1)
data.columns = ['Neighborhood', 'Latitude', 'Longitude']

In [64]:
neighborhoods_venues_sorted_all = pd.concat([neighborhoods_venues_sorted, neighborhoods_venues_sorted_msk]).reset_index().drop('index', axis=1)
neighborhoods_venues_sorted_all.head(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Железнодорожный район,Beach,Lake,Resort,Campground,Steakhouse,Bus Line,Diner,Dessert Shop,Deli / Bodega,Dance


In [74]:
data_merged = data
data_merged = data_merged.join(neighborhoods_venues_sorted_all.set_index('Neighborhood'), on='Neighborhood')
data_merged = data_merged.drop(3).reset_index().drop('index', axis=1)
data_merged['Cluster labels'] = KM.labels_


,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster labels
0,Центральный район,51.733033,39.212859,Bus Stop,Bus Line,Gym,Nature Preserve,Bistro,Beach,Recreation Center,Bus Station,Resort,River,3
1,Советский район,51.627819,39.114053,Airport,Restaurant,Bath House,Steakhouse,Bus Station,Electronics Store,Diner,Dessert Shop,Deli / Bodega,Dance,4
2,Ленинский район,51.642141,39.184415,Restaurant,Bookstore,Grocery Store,Park,Food & Drink Shop,Gym,Bakery,Arts & Crafts Store,Art Gallery,Bar,4
3,Коминтерновский район,51.718921,39.146199,Restaurant,Pub,Park,Supermarket,Bakery,Clothing Store,Electronics Store,Grocery Store,Bookstore,Gym,2
4,Железнодорожный район,51.766379,39.348958,Beach,Lake,Resort,Campground,Steakhouse,Bus Line,Diner,Dessert Shop,Deli / Bodega,Dance,0
5,Зеленоградский административный округ,55.984968,37.205612,Park,Gym,Restaurant,Bar,Coffee Shop,Café,Clothing Store,Brewery,Concert Hall,Pizza,1
6,Северо-Западный административный округ,55.826742,37.435495,Restaurant,Park,Gym,Stadium,Cosmetics Shop,Surf Spot,Beach,Shopping Mall,Pizza,Concert Hall,5
7,Западный административный округ,55.700349,37.467540,Gym,Park,Restaurant,Coffee Shop,Auto Workshop,Gourmet Shop,Spa,Sports,Health Food Store,Yoga,5
8,Юго-Западный административный округ,55.602850,37.540927,Gym,Restaurant,Coffee Shop,Clothing Store,Park,Cosmetics Shop,Auto Workshop,Gourmet Shop,Jewelry Store,Salon / Barbershop,5
9,Южный административный округ,55.647965,37.644083,Gym,Park,Health Food Store,Bar,Coffee Shop,Arcade,Butcher,Restaurant,Big Box Store,Sports,1


In [78]:

map_clusters = folium.Map(location=[54.010993, 38.290896], zoom_start=6)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighborhood'], data_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We see that there is no common clusters in Moscow and Voronezh. By using K Means I understood that these two cities are too different to generalize even only in venues field.